# **Bitcoin price prediction - Random Forest Regressor (Walk Forward Split)**
### Big Data Computing final project - A.Y. 2022 - 2023
Prof. Gabriele Tolomei

MSc in Computer Science

La Sapienza, University of Rome

### Author: Corsi Danilo (1742375) - corsi.1742375@studenti.uniroma1.it


---


Description: perform model's train / validation with hyperparameter tuning and cross validation based on different methods of splitting the dataset.

# Global constants, dependencies, libraries and tools

In [1]:
# Main constants
LOCAL_RUNNING = True
SLOW_OPERATIONS = False # Decide whether or not to use operations that might slow down notebook execution
ROOT_DIR = "D:/Documents/Repository/BDC/project" if LOCAL_RUNNING else "/content/drive"

In [2]:
if not LOCAL_RUNNING:
    # Point Colaboratory to Google Drive
    from google.colab import drive

    # Define GDrive paths
    drive.mount(ROOT_DIR, force_remount=True)

    # Install Spark and related dependencies
    !pip install pyspark
    !pip install -U -q PyDrive -qq
    !apt install openjdk-8-jdk-headless -qq

## Import my utilities

In [3]:
# Set main dir
MAIN_DIR = ROOT_DIR + "" if LOCAL_RUNNING else ROOT_DIR + "/MyDrive/BDC/project"

# Utilities dir
UTILITIES_DIR = MAIN_DIR + "/utilities"

# Import my utilities
import sys
sys.path.append(UTILITIES_DIR)

from imports import *
import train_validation_utilities
from config import *

importlib.reload(train_validation_utilities)

<module 'train_validation_utilities' from 'D:\\Documents/Repository/BDC/project/utilities\\train_validation_utilities.py'>

## Core variables

In [4]:
# BS = Block Split
# WFS = Walk Forward Split
# SS = Single Split
SPLITTING_METHOD = WFS

# LR = LinearRegression 
# GLR = GeneralizedLinearRegression 
# RF = RandomForestRegressor 
# GBTR = GradientBoostingTreeRegressor
MODEL_NAME = RF

In [5]:
###################
# --- DATASET --- #
###################

# Datasets dirs
DATASET_OUTPUT_DIR = MAIN_DIR + "/datasets/output"

# Datasets paths
DATASET_TRAIN_VALID  = DATASET_OUTPUT_DIR + "/" + DATASET_TRAIN_VALID_NAME + ".parquet"

####################
# --- FEATURES --- #
####################

# Features dir
FEATURES_DIR = MAIN_DIR + "/features"

# Features paths
FEATURES_CORRELATION = FEATURES_DIR + "/" + FEATURES_CORRELATION_LABEL + ".json"
BASE_FEATURES = FEATURES_DIR + "/" + BASE_FEATURES_LABEL + ".json"
BASE_AND_MOST_CORR_FEATURES = FEATURES_DIR + "/" + BASE_AND_MOST_CORR_FEATURES_LABEL + ".json"
BASE_AND_LEAST_CORR_FEATURES = FEATURES_DIR + "/" + BASE_AND_LEAST_CORR_FEATURES_LABEL + ".json"

##################
# --- MODELS --- #
##################

# Model dir
MODELS_DIR = MAIN_DIR + "/models"

# Model path
MODEL = MODELS_DIR + "/" + MODEL_NAME

###################
# --- RESULTS --- #
###################

# Results dir
RESULTS_DIR = MAIN_DIR + "/results/" + SPLITTING_METHOD

# Results path
ALL_MODEL_RESULTS  = RESULTS_DIR + "/" + MODEL_NAME + "_all.csv"
REL_MODEL_RESULTS  = RESULTS_DIR + "/" + MODEL_NAME + "_rel.csv"

MODEL_ACCURACY_RESULTS  = RESULTS_DIR + "/" + MODEL_NAME + "_accuracy.csv"

In [6]:
# Importing useful libraries
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pio.renderers.default = 'vscode+colab' # To correctly render plotly plots

# Create the pyspark session

In [7]:
# Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '12G').\
                set('spark.driver.memory', '12G').\
                set('spark.driver.maxResultSize', '109G').\
                set("spark.kryoserializer.buffer.max", "1G").\
                setAppName("BitcoinPricePrediction").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

# Loading dataset

In [8]:
# Load train / validation set into pyspark dataset objects
df = spark.read.load(DATASET_TRAIN_VALID,
                         format="parquet",
                         sep=",",
                         inferSchema="true",
                         header="true"
                    )

In [9]:
train_validation_utilities.dataset_info(df)

+-------------------+---+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+-----------------+--------------------+--------------------+------------------+------------------+--------------------+------------------------+-------------------+------------------+--------------------+--------------------+------------------+--------------+--------------------------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|          timestamp| id|     market-price|    opening-price|    highest-price|     lowest-price|    closing-price|  trade-volume-btc|   total-bitcoins|          market-cap|    trade-volume-usd|       blocks-size|    avg-block-size|n-transactions-total|n-transactions-per-block|          hash-rate|        difficulty|      miners-revenue|transaction-fees-usd|n-unique-addresses|n-transactions|estimated-transaction-volume-usd|       sma-5-days|   

# Loading features

In [10]:
# Loading base features
with open(BASE_FEATURES, "r") as f:
    BASE_FEATURES = json.load(f)
print(BASE_FEATURES)

['opening-price', 'highest-price', 'lowest-price', 'closing-price', 'trade-volume-btc', 'market-price', 'market-cap', 'total-bitcoins', 'trade-volume-usd']


In [11]:
# Loading currency and additional most correlated features
with open(BASE_AND_MOST_CORR_FEATURES, "r") as f:
    BASE_AND_MOST_CORR_FEATURES = json.load(f)
print(BASE_AND_MOST_CORR_FEATURES)

['opening-price', 'highest-price', 'lowest-price', 'closing-price', 'trade-volume-btc', 'market-price', 'market-cap', 'total-bitcoins', 'trade-volume-usd', 'miners-revenue', 'sma-5-days', 'sma-7-days', 'sma-10-days', 'estimated-transaction-volume-usd', 'sma-20-days']


In [12]:
# Loading currency and additional least correlated features
with open(BASE_AND_LEAST_CORR_FEATURES, "r") as f:
    BASE_AND_LEAST_CORR_FEATURES = json.load(f)
print(BASE_AND_LEAST_CORR_FEATURES)

['opening-price', 'highest-price', 'lowest-price', 'closing-price', 'trade-volume-btc', 'market-price', 'market-cap', 'total-bitcoins', 'trade-volume-usd', 'sma-100-days', 'transaction-fees-usd', 'n-unique-addresses', 'sma-50-days', 'n-transactions-total', 'blocks-size', 'hash-rate', 'difficulty', 'avg-block-size', 'n-transactions-per-block', 'n-transactions']


# Model train / validation
In order to train and validate the model I'll try several approaches:
- `Default without normalization:` make predictions using the base model
- `Default with normalization:` like the previous one but features are normalized

Then the features that gave on average the most satisfactory results (for each model) are chosen and proceeded with:
- `Hyperparameter tuning:` finding the best parameters to use. 
- `Cross Validation:` validate the performance of the model with the chosen parameters (also here using Block split / Walk forward split)

If the final results are satisfactory, the model will be trained on the whole train / validation set and saved in order to make predictions on the test set.

For each approach the train / validation set will be split according to the chosen splitting method (in order to figure out which one works best for our problem). In this case the `Walk forward time series splits` method will be used: involves using a sliding window approach to create the training and validation sets for each fold. The model is trained on a fixed window of historical data, and then validated on the next observation in the time series. This process is repeated for each subsequent observation, with the window sliding forward one step at a time. 

<img src="https://github.com/CorsiDanilo/bitcoin-price-prediction-with-pyspark/blob/main/notebooks/images/walk-forward-splits.png?raw=1">

In [13]:
# Get splitting parameters based on the choosen splitting method
splitting_info = train_validation_utilities.get_splitting_params(SPLITTING_METHOD)
splitting_info

{'split_type': 'walk_forward_splits',
 'min_obser': 20000,
 'sliding_window': 5000}

## Default
The train / validation set will be splitted based on the splitting method chosen so that the model performance can be seen without any tuning by using different features (normalized and non)

In [14]:
# Get default parameters
params = train_validation_utilities.get_defaults_model_params(MODEL_NAME)
params

{'numTrees': [20], 'maxDepth': [5], 'seed': [42]}

### Without normalization

In [15]:
# Define model and features type
MODEL_TYPE = "default"
FEATURES_NORMALIZATION = False

In [16]:
# Choose base features
CHOSEN_FEATURES = BASE_FEATURES
CHOSEN_FEATURES_LABEL = BASE_FEATURES_LABEL

In [17]:
# Make predictions by using base features
default_train_results_base_features, default_valid_results_base_features, default_train_pred_base_features, default_valid_pred_base_features = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

Split: [22/22]: 100%|██████████| 1/1 [00:01<00:00,  1.89s/it]


In [18]:
default_train_results_base_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,RandomForestRegressor,default,train,walk_forward_splits,base_features,1,"(20000, 5000)","[20, 5, 42]",54.324828,2951.186967,40.463460,0.005271,0.998088,0.998088,2.124227
1,RandomForestRegressor,default,train,walk_forward_splits,base_features,2,"(20000, 5000)","[20, 5, 42]",67.275716,4526.021944,43.766245,0.005333,0.997262,0.997261,1.000664
2,RandomForestRegressor,default,train,walk_forward_splits,base_features,3,"(20000, 5000)","[20, 5, 42]",68.267176,4660.407379,48.790814,0.005713,0.998392,0.998392,0.911856
3,RandomForestRegressor,default,train,walk_forward_splits,base_features,4,"(20000, 5000)","[20, 5, 42]",101.332426,10268.260658,63.657083,0.006096,0.996683,0.996682,0.781034
4,RandomForestRegressor,default,train,walk_forward_splits,base_features,5,"(20000, 5000)","[20, 5, 42]",245.803556,60419.387956,134.788525,0.008653,0.997781,0.997781,0.754985
5,RandomForestRegressor,default,train,walk_forward_splits,base_features,6,"(20000, 5000)","[20, 5, 42]",474.640394,225283.503611,286.217603,0.012909,0.998573,0.998572,0.713389
6,RandomForestRegressor,default,train,walk_forward_splits,base_features,7,"(20000, 5000)","[20, 5, 42]",453.399154,205570.793236,332.765242,0.012473,0.999334,0.999334,0.717053
7,RandomForestRegressor,default,train,walk_forward_splits,base_features,8,"(20000, 5000)","[20, 5, 42]",454.302951,206391.171739,300.855322,0.008288,0.998973,0.998973,0.807583
8,RandomForestRegressor,default,train,walk_forward_splits,base_features,9,"(20000, 5000)","[20, 5, 42]",345.221745,119178.052938,244.867106,0.005562,0.998812,0.998812,0.753515
9,RandomForestRegressor,default,train,walk_forward_splits,base_features,10,"(20000, 5000)","[20, 5, 42]",331.128187,109645.876060,246.937555,0.005445,0.998698,0.998698,0.730999


In [19]:
default_valid_results_base_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,RandomForestRegressor,default,valid,walk_forward_splits,base_features,1,"(20000, 5000)","[20, 5, 42]",341.014059,1.162906e+05,142.833086,0.013294,0.703295,0.703057,2.124227
1,RandomForestRegressor,default,valid,walk_forward_splits,base_features,2,"(20000, 5000)","[20, 5, 42]",451.926817,2.042378e+05,388.396665,0.034183,0.425514,0.425054,1.000664
2,RandomForestRegressor,default,valid,walk_forward_splits,base_features,3,"(20000, 5000)","[20, 5, 42]",1775.458383,3.152252e+06,1134.296754,0.078325,0.030501,0.029725,0.911856
3,RandomForestRegressor,default,valid,walk_forward_splits,base_features,4,"(20000, 5000)","[20, 5, 42]",7215.715807,5.206655e+07,5621.445609,0.233791,-1.598331,-1.600412,0.781034
4,RandomForestRegressor,default,valid,walk_forward_splits,base_features,5,"(20000, 5000)","[20, 5, 42]",12913.689053,1.667634e+08,10528.753877,0.235727,-2.051569,-2.054013,0.754985
5,RandomForestRegressor,default,valid,walk_forward_splits,base_features,6,"(20000, 5000)","[20, 5, 42]",3023.558357,9.141905e+06,2332.379703,0.040161,0.516360,0.515973,0.713389
6,RandomForestRegressor,default,valid,walk_forward_splits,base_features,7,"(20000, 5000)","[20, 5, 42]",1810.830679,3.279108e+06,1406.283847,0.035560,0.957918,0.957885,0.717053
7,RandomForestRegressor,default,valid,walk_forward_splits,base_features,8,"(20000, 5000)","[20, 5, 42]",596.276765,3.555460e+05,448.545050,0.012828,0.958328,0.958295,0.807583
8,RandomForestRegressor,default,valid,walk_forward_splits,base_features,9,"(20000, 5000)","[20, 5, 42]",710.370482,5.046262e+05,523.555803,0.011474,0.940783,0.940735,0.753515
9,RandomForestRegressor,default,valid,walk_forward_splits,base_features,10,"(20000, 5000)","[20, 5, 42]",1708.787772,2.919956e+06,1010.541397,0.016595,0.950629,0.950590,0.730999


In [20]:
# Choose base and most additional correlated features
CHOSEN_FEATURES = BASE_AND_MOST_CORR_FEATURES
CHOSEN_FEATURES_LABEL = BASE_AND_MOST_CORR_FEATURES_LABEL

In [21]:
# Make predictions by using base and most additional correlated features
default_train_results_base_and_most_corr_features, default_valid_results_base_and_most_corr_features, default_train_pred_base_and_most_corr_features, default_valid_pred_base_and_most_corr_features = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

Split: [22/22]: 100%|██████████| 1/1 [00:01<00:00,  1.92s/it]


In [22]:
default_train_results_base_and_most_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,RandomForestRegressor,default,train,walk_forward_splits,base_and_most_corr_features,1,"(20000, 5000)","[20, 5, 42]",55.884373,3123.063151,42.855391,0.005617,0.997977,0.997977,0.800778
1,RandomForestRegressor,default,train,walk_forward_splits,base_and_most_corr_features,2,"(20000, 5000)","[20, 5, 42]",54.077732,2924.401054,38.347669,0.004709,0.998231,0.998230,0.755819
2,RandomForestRegressor,default,train,walk_forward_splits,base_and_most_corr_features,3,"(20000, 5000)","[20, 5, 42]",65.590702,4302.140213,46.501029,0.005458,0.998516,0.998515,0.735129
3,RandomForestRegressor,default,train,walk_forward_splits,base_and_most_corr_features,4,"(20000, 5000)","[20, 5, 42]",87.214075,7606.294795,56.646746,0.005313,0.997543,0.997542,0.799385
4,RandomForestRegressor,default,train,walk_forward_splits,base_and_most_corr_features,5,"(20000, 5000)","[20, 5, 42]",208.425711,43441.277030,120.940702,0.008069,0.998405,0.998404,0.741255
5,RandomForestRegressor,default,train,walk_forward_splits,base_and_most_corr_features,6,"(20000, 5000)","[20, 5, 42]",421.601079,177747.469841,273.053898,0.012779,0.998874,0.998874,0.765357
6,RandomForestRegressor,default,train,walk_forward_splits,base_and_most_corr_features,7,"(20000, 5000)","[20, 5, 42]",435.040829,189260.522983,315.295167,0.011953,0.999387,0.999387,0.731522
7,RandomForestRegressor,default,train,walk_forward_splits,base_and_most_corr_features,8,"(20000, 5000)","[20, 5, 42]",445.454478,198429.691872,303.619560,0.008393,0.999013,0.999012,0.763512
8,RandomForestRegressor,default,train,walk_forward_splits,base_and_most_corr_features,9,"(20000, 5000)","[20, 5, 42]",334.790056,112084.381733,243.431235,0.005631,0.998883,0.998883,0.804514
9,RandomForestRegressor,default,train,walk_forward_splits,base_and_most_corr_features,10,"(20000, 5000)","[20, 5, 42]",316.341216,100071.764904,236.004004,0.005233,0.998812,0.998811,0.752996


In [23]:
default_valid_results_base_and_most_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,RandomForestRegressor,default,valid,walk_forward_splits,base_and_most_corr_features,1,"(20000, 5000)","[20, 5, 42]",380.676138,1.449143e+05,164.197890,0.015358,0.630264,0.629968,0.800778
1,RandomForestRegressor,default,valid,walk_forward_splits,base_and_most_corr_features,2,"(20000, 5000)","[20, 5, 42]",491.449652,2.415228e+05,401.108825,0.036253,0.320638,0.320094,0.755819
2,RandomForestRegressor,default,valid,walk_forward_splits,base_and_most_corr_features,3,"(20000, 5000)","[20, 5, 42]",1812.134802,3.283833e+06,1169.295549,0.080938,-0.009967,-0.010776,0.735129
3,RandomForestRegressor,default,valid,walk_forward_splits,base_and_most_corr_features,4,"(20000, 5000)","[20, 5, 42]",7247.268773,5.252290e+07,5663.362098,0.235824,-1.621105,-1.623204,0.799385
4,RandomForestRegressor,default,valid,walk_forward_splits,base_and_most_corr_features,5,"(20000, 5000)","[20, 5, 42]",12716.236660,1.617027e+08,10410.077041,0.233592,-1.958965,-1.961334,0.741255
5,RandomForestRegressor,default,valid,walk_forward_splits,base_and_most_corr_features,6,"(20000, 5000)","[20, 5, 42]",2754.285142,7.586087e+06,2098.132145,0.036213,0.598669,0.598347,0.765357
6,RandomForestRegressor,default,valid,walk_forward_splits,base_and_most_corr_features,7,"(20000, 5000)","[20, 5, 42]",2630.659771,6.920371e+06,1944.358582,0.050280,0.911189,0.911118,0.731522
7,RandomForestRegressor,default,valid,walk_forward_splits,base_and_most_corr_features,8,"(20000, 5000)","[20, 5, 42]",605.009533,3.660365e+05,442.072271,0.012520,0.957098,0.957064,0.763512
8,RandomForestRegressor,default,valid,walk_forward_splits,base_and_most_corr_features,9,"(20000, 5000)","[20, 5, 42]",568.970865,3.237278e+05,430.923664,0.009418,0.962011,0.961980,0.804514
9,RandomForestRegressor,default,valid,walk_forward_splits,base_and_most_corr_features,10,"(20000, 5000)","[20, 5, 42]",2004.584149,4.018358e+06,1201.484097,0.019509,0.932057,0.932003,0.752996


In [24]:
# Choose base and least additional correlated features
CHOSEN_FEATURES = BASE_AND_LEAST_CORR_FEATURES
CHOSEN_FEATURES_LABEL = BASE_AND_LEAST_CORR_FEATURES_LABEL

In [25]:
# Make predictions by using base and least additional correlated features
default_train_results_base_and_least_corr_features, default_valid_results_base_and_least_corr_features, default_train_pred_base_and_least_corr_features, default_valid_pred_base_and_least_corr_features = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

Split: [22/22]: 100%|██████████| 1/1 [00:02<00:00,  2.01s/it]


In [26]:
default_train_results_base_and_least_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,RandomForestRegressor,default,train,walk_forward_splits,base_and_least_corr_features,1,"(20000, 5000)","[20, 5, 42]",55.536725,3084.327784,40.765889,0.005364,0.998002,0.998002,0.909670
1,RandomForestRegressor,default,train,walk_forward_splits,base_and_least_corr_features,2,"(20000, 5000)","[20, 5, 42]",53.499846,2862.233507,37.313675,0.004631,0.998268,0.998268,0.845103
2,RandomForestRegressor,default,train,walk_forward_splits,base_and_least_corr_features,3,"(20000, 5000)","[20, 5, 42]",64.840827,4204.332807,47.165961,0.005495,0.998549,0.998549,0.871024
3,RandomForestRegressor,default,train,walk_forward_splits,base_and_least_corr_features,4,"(20000, 5000)","[20, 5, 42]",79.373063,6300.083141,51.674075,0.004946,0.997965,0.997964,0.948189
4,RandomForestRegressor,default,train,walk_forward_splits,base_and_least_corr_features,5,"(20000, 5000)","[20, 5, 42]",157.170225,24702.479639,103.755659,0.007309,0.999093,0.999093,0.817850
5,RandomForestRegressor,default,train,walk_forward_splits,base_and_least_corr_features,6,"(20000, 5000)","[20, 5, 42]",376.797037,141976.007101,252.283889,0.012051,0.999101,0.999100,0.913142
6,RandomForestRegressor,default,train,walk_forward_splits,base_and_least_corr_features,7,"(20000, 5000)","[20, 5, 42]",424.936019,180570.619959,304.753420,0.011261,0.999415,0.999415,0.903635
7,RandomForestRegressor,default,train,walk_forward_splits,base_and_least_corr_features,8,"(20000, 5000)","[20, 5, 42]",428.810983,183878.859499,290.086866,0.007889,0.999085,0.999085,0.865193
8,RandomForestRegressor,default,train,walk_forward_splits,base_and_least_corr_features,9,"(20000, 5000)","[20, 5, 42]",334.172402,111671.194392,234.756466,0.005423,0.998887,0.998887,0.894993
9,RandomForestRegressor,default,train,walk_forward_splits,base_and_least_corr_features,10,"(20000, 5000)","[20, 5, 42]",305.888803,93567.959527,229.348511,0.005102,0.998889,0.998889,0.914337


In [27]:
default_valid_results_base_and_least_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,RandomForestRegressor,default,valid,walk_forward_splits,base_and_least_corr_features,1,"(20000, 5000)","[20, 5, 42]",350.633123,1.229436e+05,146.548859,0.013642,0.686320,0.686069,0.909670
1,RandomForestRegressor,default,valid,walk_forward_splits,base_and_least_corr_features,2,"(20000, 5000)","[20, 5, 42]",520.840849,2.712752e+05,429.487927,0.038004,0.236950,0.236339,0.845103
2,RandomForestRegressor,default,valid,walk_forward_splits,base_and_least_corr_features,3,"(20000, 5000)","[20, 5, 42]",1792.313674,3.212388e+06,1136.756930,0.078282,0.012006,0.011215,0.871024
3,RandomForestRegressor,default,valid,walk_forward_splits,base_and_least_corr_features,4,"(20000, 5000)","[20, 5, 42]",7810.503835,6.100397e+07,6357.850351,0.269305,-2.044344,-2.046782,0.948189
4,RandomForestRegressor,default,valid,walk_forward_splits,base_and_least_corr_features,5,"(20000, 5000)","[20, 5, 42]",12507.577521,1.564395e+08,10026.930335,0.223138,-1.862655,-1.864947,0.817850
5,RandomForestRegressor,default,valid,walk_forward_splits,base_and_least_corr_features,6,"(20000, 5000)","[20, 5, 42]",3288.721000,1.081569e+07,2640.325653,0.045472,0.427812,0.427353,0.913142
6,RandomForestRegressor,default,valid,walk_forward_splits,base_and_least_corr_features,7,"(20000, 5000)","[20, 5, 42]",3163.783466,1.000953e+07,2262.212466,0.059233,0.871545,0.871442,0.903635
7,RandomForestRegressor,default,valid,walk_forward_splits,base_and_least_corr_features,8,"(20000, 5000)","[20, 5, 42]",653.560038,4.271407e+05,443.057026,0.012306,0.949937,0.949896,0.865193
8,RandomForestRegressor,default,valid,walk_forward_splits,base_and_least_corr_features,9,"(20000, 5000)","[20, 5, 42]",786.144424,6.180231e+05,666.370743,0.014147,0.927476,0.927417,0.894993
9,RandomForestRegressor,default,valid,walk_forward_splits,base_and_least_corr_features,10,"(20000, 5000)","[20, 5, 42]",2584.241012,6.678302e+06,1780.736451,0.028774,0.887083,0.886992,0.914337


### With normalization

In [28]:
# Define model and features type
MODEL_TYPE = "default_norm"
FEATURES_NORMALIZATION = True

In [29]:
# Choose base features
CHOSEN_FEATURES = BASE_FEATURES
CHOSEN_FEATURES_LABEL = BASE_FEATURES_LABEL

In [30]:
# Make predictions by using base features
default_norm_train_results_base_features, default_norm_valid_results_base_features, default_norm_train_pred_base_features, default_norm_valid_pred_base_features = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

Split: [22/22]: 100%|██████████| 1/1 [00:01<00:00,  1.84s/it]


In [31]:
default_norm_train_results_base_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,RandomForestRegressor,default_norm,train,walk_forward_splits,base_features_norm,1,"(20000, 5000)","[20, 5, 42]",262.475778,6.889353e+04,207.602294,0.025876,0.955373,0.955364,0.723095
1,RandomForestRegressor,default_norm,train,walk_forward_splits,base_features_norm,2,"(20000, 5000)","[20, 5, 42]",268.133114,7.189537e+04,197.585347,0.023598,0.956503,0.956495,0.796232
2,RandomForestRegressor,default_norm,train,walk_forward_splits,base_features_norm,3,"(20000, 5000)","[20, 5, 42]",313.707297,9.841227e+04,236.576907,0.027473,0.966044,0.966037,0.707100
3,RandomForestRegressor,default_norm,train,walk_forward_splits,base_features_norm,4,"(20000, 5000)","[20, 5, 42]",376.201026,1.415272e+05,278.469729,0.027531,0.954277,0.954268,0.708658
4,RandomForestRegressor,default_norm,train,walk_forward_splits,base_features_norm,5,"(20000, 5000)","[20, 5, 42]",787.265667,6.197872e+05,612.104458,0.047886,0.977240,0.977235,0.762123
5,RandomForestRegressor,default_norm,train,walk_forward_splits,base_features_norm,6,"(20000, 5000)","[20, 5, 42]",1594.855164,2.543563e+06,1175.594148,0.063857,0.983885,0.983882,0.808950
6,RandomForestRegressor,default_norm,train,walk_forward_splits,base_features_norm,7,"(20000, 5000)","[20, 5, 42]",2087.725793,4.358599e+06,1595.527515,0.067536,0.985887,0.985884,0.738198
7,RandomForestRegressor,default_norm,train,walk_forward_splits,base_features_norm,8,"(20000, 5000)","[20, 5, 42]",2296.707285,5.274864e+06,1813.232405,0.057877,0.973753,0.973748,0.797259
8,RandomForestRegressor,default_norm,train,walk_forward_splits,base_features_norm,9,"(20000, 5000)","[20, 5, 42]",1626.790707,2.646448e+06,1219.785241,0.028649,0.973623,0.973617,0.776791
9,RandomForestRegressor,default_norm,train,walk_forward_splits,base_features_norm,10,"(20000, 5000)","[20, 5, 42]",1488.242263,2.214865e+06,1120.080895,0.025976,0.973699,0.973693,0.762012


In [32]:
default_norm_valid_results_base_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,RandomForestRegressor,default_norm,valid,walk_forward_splits,base_features_norm,1,"(20000, 5000)","[20, 5, 42]",645.698275,4.169263e+05,340.093045,0.032686,-0.063751,-0.064603,0.723095
1,RandomForestRegressor,default_norm,valid,walk_forward_splits,base_features_norm,2,"(20000, 5000)","[20, 5, 42]",1268.262192,1.608489e+06,1170.996230,0.102545,-3.524401,-3.528024,0.796232
2,RandomForestRegressor,default_norm,valid,walk_forward_splits,base_features_norm,3,"(20000, 5000)","[20, 5, 42]",2280.163967,5.199148e+06,1526.211225,0.106598,-0.599037,-0.600317,0.707100
3,RandomForestRegressor,default_norm,valid,walk_forward_splits,base_features_norm,4,"(20000, 5000)","[20, 5, 42]",8288.886669,6.870564e+07,7013.733126,0.302384,-2.428689,-2.431434,0.708658
4,RandomForestRegressor,default_norm,valid,walk_forward_splits,base_features_norm,5,"(20000, 5000)","[20, 5, 42]",16184.718790,2.619451e+08,13697.566024,0.312613,-3.793281,-3.797120,0.762123
5,RandomForestRegressor,default_norm,valid,walk_forward_splits,base_features_norm,6,"(20000, 5000)","[20, 5, 42]",12100.744230,1.464280e+08,11064.020676,0.193054,-6.746566,-6.752769,0.808950
6,RandomForestRegressor,default_norm,valid,walk_forward_splits,base_features_norm,7,"(20000, 5000)","[20, 5, 42]",3169.035368,1.004279e+07,2429.132392,0.052804,0.871118,0.871015,0.738198
7,RandomForestRegressor,default_norm,valid,walk_forward_splits,base_features_norm,8,"(20000, 5000)","[20, 5, 42]",3191.031908,1.018268e+07,2673.191752,0.075565,-0.193472,-0.194427,0.797259
8,RandomForestRegressor,default_norm,valid,walk_forward_splits,base_features_norm,9,"(20000, 5000)","[20, 5, 42]",2994.384779,8.966340e+06,2575.890230,0.056045,-0.052192,-0.053035,0.776791
9,RandomForestRegressor,default_norm,valid,walk_forward_splits,base_features_norm,10,"(20000, 5000)","[20, 5, 42]",5681.285322,3.227700e+07,4629.849051,0.076541,0.454257,0.453820,0.762012


In [33]:
# Choose base and most additional correlated features
CHOSEN_FEATURES = BASE_AND_MOST_CORR_FEATURES
CHOSEN_FEATURES_LABEL = BASE_AND_MOST_CORR_FEATURES_LABEL

In [34]:
# Make predictions by using base and most additional correlated features
default_norm_train_results_base_and_most_corr_features, default_norm_valid_results_base_and_most_corr_features, default_norm_train_pred_base_and_most_corr_features, default_norm_valid_pred_base_and_most_corr_features = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

Split: [22/22]: 100%|██████████| 1/1 [00:02<00:00,  2.06s/it]


In [35]:
default_norm_train_results_base_and_most_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,RandomForestRegressor,default_norm,train,walk_forward_splits,base_and_most_corr_features_norm,1,"(20000, 5000)","[20, 5, 42]",133.823537,1.790874e+04,105.771006,0.013438,0.988399,0.988397,0.850189
1,RandomForestRegressor,default_norm,train,walk_forward_splits,base_and_most_corr_features_norm,2,"(20000, 5000)","[20, 5, 42]",126.055437,1.588997e+04,93.034999,0.011111,0.990387,0.990385,0.786912
2,RandomForestRegressor,default_norm,train,walk_forward_splits,base_and_most_corr_features_norm,3,"(20000, 5000)","[20, 5, 42]",132.213744,1.748047e+04,97.114165,0.010986,0.993969,0.993967,0.778359
3,RandomForestRegressor,default_norm,train,walk_forward_splits,base_and_most_corr_features_norm,4,"(20000, 5000)","[20, 5, 42]",149.643573,2.239320e+04,111.716320,0.010658,0.992766,0.992764,0.780360
4,RandomForestRegressor,default_norm,train,walk_forward_splits,base_and_most_corr_features_norm,5,"(20000, 5000)","[20, 5, 42]",230.734680,5.323849e+04,171.969806,0.013055,0.998045,0.998045,0.873467
5,RandomForestRegressor,default_norm,train,walk_forward_splits,base_and_most_corr_features_norm,6,"(20000, 5000)","[20, 5, 42]",669.018263,4.475854e+05,429.028190,0.018437,0.997164,0.997164,0.790060
6,RandomForestRegressor,default_norm,train,walk_forward_splits,base_and_most_corr_features_norm,7,"(20000, 5000)","[20, 5, 42]",1334.676854,1.781362e+06,908.951745,0.025937,0.994232,0.994231,0.846836
7,RandomForestRegressor,default_norm,train,walk_forward_splits,base_and_most_corr_features_norm,8,"(20000, 5000)","[20, 5, 42]",1460.559295,2.133233e+06,1094.549316,0.026977,0.989385,0.989383,0.802643
8,RandomForestRegressor,default_norm,train,walk_forward_splits,base_and_most_corr_features_norm,9,"(20000, 5000)","[20, 5, 42]",1398.660409,1.956251e+06,1020.451118,0.022555,0.980502,0.980498,0.812807
9,RandomForestRegressor,default_norm,train,walk_forward_splits,base_and_most_corr_features_norm,10,"(20000, 5000)","[20, 5, 42]",905.110383,8.192248e+05,630.109988,0.013553,0.990272,0.990270,0.828047


In [36]:
default_norm_valid_results_base_and_most_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,RandomForestRegressor,default_norm,valid,walk_forward_splits,base_and_most_corr_features_norm,1,"(20000, 5000)","[20, 5, 42]",498.661324,2.486631e+05,366.323439,0.036887,0.365558,0.365050,0.850189
1,RandomForestRegressor,default_norm,valid,walk_forward_splits,base_and_most_corr_features_norm,2,"(20000, 5000)","[20, 5, 42]",900.539437,8.109713e+05,828.730599,0.074939,-1.281122,-1.282948,0.786912
2,RandomForestRegressor,default_norm,valid,walk_forward_splits,base_and_most_corr_features_norm,3,"(20000, 5000)","[20, 5, 42]",1928.848100,3.720455e+06,1380.997394,0.098427,-0.144254,-0.145170,0.778359
3,RandomForestRegressor,default_norm,valid,walk_forward_splits,base_and_most_corr_features_norm,4,"(20000, 5000)","[20, 5, 42]",7472.753002,5.584204e+07,5964.020865,0.250578,-1.786743,-1.788975,0.780360
4,RandomForestRegressor,default_norm,valid,walk_forward_splits,base_and_most_corr_features_norm,5,"(20000, 5000)","[20, 5, 42]",13859.523958,1.920864e+08,12133.741289,0.281560,-2.514951,-2.517765,0.873467
5,RandomForestRegressor,default_norm,valid,walk_forward_splits,base_and_most_corr_features_norm,6,"(20000, 5000)","[20, 5, 42]",16840.501343,2.836025e+08,15041.548512,0.261823,-14.003587,-14.015601,0.790060
6,RandomForestRegressor,default_norm,valid,walk_forward_splits,base_and_most_corr_features_norm,7,"(20000, 5000)","[20, 5, 42]",20929.369196,4.380385e+08,20409.192624,0.450357,-4.621465,-4.625966,0.846836
7,RandomForestRegressor,default_norm,valid,walk_forward_splits,base_and_most_corr_features_norm,8,"(20000, 5000)","[20, 5, 42]",2944.021169,8.667261e+06,2495.188468,0.074172,-0.015855,-0.016668,0.802643
8,RandomForestRegressor,default_norm,valid,walk_forward_splits,base_and_most_corr_features_norm,9,"(20000, 5000)","[20, 5, 42]",1449.845986,2.102053e+06,1135.865548,0.024664,0.753326,0.753128,0.812807
9,RandomForestRegressor,default_norm,valid,walk_forward_splits,base_and_most_corr_features_norm,10,"(20000, 5000)","[20, 5, 42]",5762.090387,3.320169e+07,4577.339607,0.073955,0.438622,0.438173,0.828047


In [37]:
# Choose base and least additional correlated features
CHOSEN_FEATURES = BASE_AND_LEAST_CORR_FEATURES
CHOSEN_FEATURES_LABEL = BASE_AND_LEAST_CORR_FEATURES_LABEL

In [38]:
# Make predictions by using base and least additional correlated features
default_norm_train_results_base_and_least_corr_features, default_norm_valid_results_base_and_least_corr_features, default_norm_train_pred_base_and_least_corr_features, default_norm_valid_pred_base_and_least_corr_features = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

Split: [22/22]: 100%|██████████| 1/1 [00:02<00:00,  2.19s/it]


In [39]:
default_norm_train_results_base_and_least_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,RandomForestRegressor,default_norm,train,walk_forward_splits,base_and_least_corr_features_norm,1,"(20000, 5000)","[20, 5, 42]",135.667828,1.840576e+04,103.343535,0.012910,0.988077,0.988075,0.859265
1,RandomForestRegressor,default_norm,train,walk_forward_splits,base_and_least_corr_features_norm,2,"(20000, 5000)","[20, 5, 42]",154.204799,2.377912e+04,115.978371,0.013532,0.985614,0.985611,0.911677
2,RandomForestRegressor,default_norm,train,walk_forward_splits,base_and_least_corr_features_norm,3,"(20000, 5000)","[20, 5, 42]",135.086438,1.824835e+04,102.512346,0.011572,0.993704,0.993702,0.873301
3,RandomForestRegressor,default_norm,train,walk_forward_splits,base_and_least_corr_features_norm,4,"(20000, 5000)","[20, 5, 42]",187.466690,3.514376e+04,145.382442,0.014159,0.988646,0.988644,0.860543
4,RandomForestRegressor,default_norm,train,walk_forward_splits,base_and_least_corr_features_norm,5,"(20000, 5000)","[20, 5, 42]",242.319576,5.871878e+04,174.952606,0.013133,0.997844,0.997843,0.837170
5,RandomForestRegressor,default_norm,train,walk_forward_splits,base_and_least_corr_features_norm,6,"(20000, 5000)","[20, 5, 42]",478.254366,2.287272e+05,340.985050,0.017531,0.998551,0.998551,0.906071
6,RandomForestRegressor,default_norm,train,walk_forward_splits,base_and_least_corr_features_norm,7,"(20000, 5000)","[20, 5, 42]",595.259813,3.543342e+05,429.044815,0.014656,0.998853,0.998852,0.864639
7,RandomForestRegressor,default_norm,train,walk_forward_splits,base_and_least_corr_features_norm,8,"(20000, 5000)","[20, 5, 42]",846.222812,7.160930e+05,608.391226,0.015726,0.996437,0.996436,0.925980
8,RandomForestRegressor,default_norm,train,walk_forward_splits,base_and_least_corr_features_norm,9,"(20000, 5000)","[20, 5, 42]",813.140450,6.611974e+05,609.222383,0.014284,0.993410,0.993408,0.877642
9,RandomForestRegressor,default_norm,train,walk_forward_splits,base_and_least_corr_features_norm,10,"(20000, 5000)","[20, 5, 42]",1198.666481,1.436801e+06,838.938954,0.019304,0.982938,0.982935,0.866321


In [40]:
default_norm_valid_results_base_and_least_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,RandomForestRegressor,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,1,"(20000, 5000)","[20, 5, 42]",589.168097,3.471190e+05,464.543445,0.047113,0.114356,0.113647,0.859265
1,RandomForestRegressor,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,2,"(20000, 5000)","[20, 5, 42]",1016.644694,1.033566e+06,919.521954,0.082223,-1.907243,-1.909571,0.911677
2,RandomForestRegressor,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,3,"(20000, 5000)","[20, 5, 42]",2776.829342,7.710781e+06,1932.292582,0.137083,-1.371508,-1.373407,0.873301
3,RandomForestRegressor,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,4,"(20000, 5000)","[20, 5, 42]",7415.759641,5.499349e+07,5818.047383,0.242650,-1.744397,-1.746595,0.860543
4,RandomForestRegressor,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,5,"(20000, 5000)","[20, 5, 42]",14074.210412,1.980834e+08,11937.351065,0.271767,-2.624688,-2.627591,0.837170
5,RandomForestRegressor,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,6,"(20000, 5000)","[20, 5, 42]",3680.961508,1.354948e+07,3078.028630,0.053385,0.283184,0.282610,0.906071
6,RandomForestRegressor,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,7,"(20000, 5000)","[20, 5, 42]",3631.305471,1.318638e+07,3029.355049,0.070345,0.830776,0.830640,0.864639
7,RandomForestRegressor,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,8,"(20000, 5000)","[20, 5, 42]",12880.966405,1.659193e+08,10018.946810,0.289909,-18.446735,-18.462308,0.925980
8,RandomForestRegressor,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,9,"(20000, 5000)","[20, 5, 42]",8888.313186,7.900211e+07,8198.500781,0.174457,-8.270830,-8.278254,0.877642
9,RandomForestRegressor,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,10,"(20000, 5000)","[20, 5, 42]",11070.929504,1.225655e+08,10030.519608,0.168531,-1.072351,-1.074011,0.866321


In [41]:
# Define model information and evaluators to show
model_info = ['Model', 'Type', 'Dataset', 'Splitting', 'Features', 'Parameters']
evaluator_lst = ['RMSE', 'MSE', 'MAE', 'MAPE', 'R2', 'Adjusted_R2', 'Time']

In [42]:
# Define the results to show
default_comparison_lst = [default_valid_results_base_features, default_valid_results_base_and_most_corr_features, default_valid_results_base_and_least_corr_features, default_norm_valid_results_base_features, default_norm_valid_results_base_and_most_corr_features, default_norm_valid_results_base_and_least_corr_features]

# Show the comparison table
default_comparison_lst_df = pd.concat([train_validation_utilities.model_comparison(results, model_info, evaluator_lst) for results in default_comparison_lst])
default_comparison_lst_df

,Model,Type,Dataset,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,RandomForestRegressor,default,valid,walk_forward_splits,base_features,"[20, 5, 42]",2370.519970,1.560697e+07,1896.375868,0.065433,-0.305707,-0.306753,0.835004
0,RandomForestRegressor,default,valid,walk_forward_splits,base_and_most_corr_features,"[20, 5, 42]",2478.345441,1.576778e+07,1990.447497,0.068993,-0.408750,-0.409878,0.770615
0,RandomForestRegressor,default,valid,walk_forward_splits,base_and_least_corr_features,"[20, 5, 42]",2648.102198,1.744722e+07,2142.194716,0.073721,-0.655070,-0.656395,0.888322
0,RandomForestRegressor,default_norm,valid,walk_forward_splits,base_features_norm,"[20, 5, 42]",4311.414051,3.407697e+07,3717.619259,0.121903,-5.011666,-5.016480,0.750290
0,RandomForestRegressor,default_norm,valid,walk_forward_splits,base_and_most_corr_features_norm,"[20, 5, 42]",5185.701605,5.522636e+07,4625.144485,0.138864,-5.399166,-5.404291,0.825756
0,RandomForestRegressor,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,"[20, 5, 42]",4707.588012,3.835252e+07,4032.876095,0.124155,-2.599675,-2.602557,0.906167


In [43]:
# Save the best default model results and predicitons
best_default_results = pd.concat([train_validation_utilities.model_comparison(results, model_info, evaluator_lst) for results in [default_valid_results_base_features]])
best_default_predictions = default_valid_pred_base_features
best_default_results

,Model,Type,Dataset,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,RandomForestRegressor,default,valid,walk_forward_splits,base_features,"[20, 5, 42]",2370.51997,1.560697e+07,1896.375868,0.065433,-0.305707,-0.306753,0.835004


In [44]:
# Define the choosen features
FEATURES_NORMALIZATION = False
CHOSEN_FEATURES = BASE_FEATURES
CHOSEN_FEATURES_LABEL = BASE_FEATURES_LABEL

## Tuned
Once the features and execution method are selected, the model will undergo hyperparameter tuning and cross validation to find the best configuration. 

In [45]:
# Get model grid parameters
params = train_validation_utilities.get_model_grid_params(MODEL_NAME)
params

{'numTrees': [3, 5, 10, 20, 30], 'maxDepth': [3, 5, 10], 'seed': [42]}

### Hyperparameter tuning

In [46]:
# Select the type of feature to be used
MODEL_TYPE = "hyp_tuning"

In [47]:
# Perform hyperparameter tuning
hyp_res = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)
hyp_res

Split: [1/22]: 100%|██████████| 15/15 [00:25<00:00,  1.71s/it]


Best parameters chosen for split [1/22]: [3, 10, 42]


Split: [2/22]: 100%|██████████| 15/15 [00:26<00:00,  1.76s/it]


Best parameters chosen for split [2/22]: [3, 10, 42]


Split: [3/22]: 100%|██████████| 15/15 [00:25<00:00,  1.67s/it]


...

Split: [21/22]: 100%|██████████| 15/15 [00:24<00:00,  1.66s/it]


Best parameters chosen for split [21/22]: [20, 5, 42]


Split: [22/22]: 100%|██████████| 15/15 [00:24<00:00,  1.64s/it]

Best parameters chosen for split [22/22]: [10, 10, 42]


,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,RandomForestRegressor,hyp_tuning,valid,walk_forward_splits,base_features,1,"(20000, 5000)","[3, 10, 42]",314.768648,9.907930e+04,132.938768,0.012410,0.747208,0.747005,0.695701
1,RandomForestRegressor,hyp_tuning,valid,walk_forward_splits,base_features,2,"(20000, 5000)","[3, 10, 42]",400.353365,1.602828e+05,334.683212,0.029451,0.549152,0.548791,0.757241
2,RandomForestRegressor,hyp_tuning,valid,walk_forward_splits,base_features,3,"(20000, 5000)","[3, 10, 42]",1629.767381,2.656142e+06,1009.015574,0.069176,0.183084,0.182430,0.685774
3,RandomForestRegressor,hyp_tuning,valid,walk_forward_splits,base_features,4,"(20000, 5000)","[3, 10, 42]",7036.130720,4.950714e+07,5458.946267,0.226675,-1.470606,-1.472584,0.706308
4,RandomForestRegressor,hyp_tuning,valid,walk_forward_splits,base_features,5,"(20000, 5000)","[20, 3, 42]",12363.714181,1.528614e+08,9911.776051,0.220214,-1.797181,-1.799421,0.397253
5,RandomForestRegressor,hyp_tuning,valid,walk_forward_splits,base_features,6,"(20000, 5000)","[3, 10, 42]",2553.021163,6.517917e+06,1913.161875,0.033209,0.655179,0.654903,0.685237
6,RandomForestRegressor,hyp_tuning,valid,walk_forward_splits,base_features,7,"(20000, 5000)","[10, 10, 42]",1372.244856,1.883056e+06,1077.118627,0.026814,0.975834,0.975815,0.974319
7,RandomForestRegressor,hyp_tuning,valid,walk_forward_splits,base_features,8,"(20000, 5000)","[5, 10, 42]",482.778963,2.330755e+05,348.615921,0.009799,0.972682,0.972660,0.730780
8,RandomForestRegressor,hyp_tuning,valid,walk_forward_splits,base_features,9,"(20000, 5000)","[30, 10, 42]",577.376055,3.333631e+05,431.779274,0.009469,0.960880,0.960849,2.379147
9,RandomForestRegressor,hyp_tuning,valid,walk_forward_splits,base_features,10,"(20000, 5000)","[5, 5, 42]",1529.544155,2.339505e+06,994.824313,0.016614,0.960443,0.960412,0.369313


---
Since during this stage will be used the Block split / Walk forward split method of the dataset I compute a score for each parameter chosen by each split, assigning weights based on:
   * Their `frequency` for each split (if the same parameters are chosen from several splits, these will have greater weight) 
   * The `split` they belong to (the closer the split is to today's date the more weight they will have)
   * Their `RMSE value` for each split (the lower this is, the more weight they will have)
   
   Then, the overall score will be calculated by putting together these three weights for each parameter and the one with the best score will be the chosen parameter.

In [48]:
# Show parameters score
grouped_scores, best_params = train_validation_utilities.choose_best_params(hyp_res)
grouped_scores

,Split weight,RMSE weight,Frequency weight,Final score
Parameters,,,,
"(10, 10, 42)",0.709091,0.896228,0.833333,0.529589
"(30, 10, 42)",0.621212,0.970618,0.500000,0.301480
"(3, 10, 42)",0.257576,0.834291,1.000000,0.214893
"(5, 10, 42)",0.515152,0.766968,0.500000,0.197552
"(20, 5, 42)",0.954545,0.953261,0.166667,0.151655
"(5, 5, 42)",0.545455,0.725263,0.333333,0.131866
"(30, 5, 42)",0.727273,0.975826,0.166667,0.118282
"(20, 3, 42)",0.227273,0.000000,0.166667,0.000000


In [49]:
# Print best parameters
print(f"Best parameters: {best_params}")

Best parameters: (10, 10, 42)


### Cross validation

In [50]:
MODEL_TYPE = "cross_val"

In [51]:
# Get tuned parameters
params = train_validation_utilities.get_best_model_params(best_params, MODEL_NAME)
params

{'numTrees': [10], 'maxDepth': [10], 'seed': [42]}

In [52]:
# Perform cross validation
cv_train_result, cv_valid_result, cv_train_pred, cv_valid_pred = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

Split: [22/22]: 100%|██████████| 1/1 [00:02<00:00,  2.22s/it]


In [53]:
cv_train_result

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,RandomForestRegressor,cross_val,train,walk_forward_splits,base_features,1,"(20000, 5000)","[10, 10, 42]",21.875971,478.558112,13.902996,0.001820,0.999690,0.999690,1.156645
1,RandomForestRegressor,cross_val,train,walk_forward_splits,base_features,2,"(20000, 5000)","[10, 10, 42]",35.190901,1238.399515,16.934609,0.002011,0.999251,0.999251,1.262350
2,RandomForestRegressor,cross_val,train,walk_forward_splits,base_features,3,"(20000, 5000)","[10, 10, 42]",23.111129,534.124297,15.144886,0.001814,0.999816,0.999816,1.141181
3,RandomForestRegressor,cross_val,train,walk_forward_splits,base_features,4,"(20000, 5000)","[10, 10, 42]",43.173344,1863.937610,21.559442,0.002114,0.999398,0.999398,1.125212
4,RandomForestRegressor,cross_val,train,walk_forward_splits,base_features,5,"(20000, 5000)","[10, 10, 42]",219.615814,48231.105873,70.126146,0.003423,0.998229,0.998228,1.040986
5,RandomForestRegressor,cross_val,train,walk_forward_splits,base_features,6,"(20000, 5000)","[10, 10, 42]",165.758818,27475.985828,80.621651,0.003169,0.999826,0.999826,1.084377
6,RandomForestRegressor,cross_val,train,walk_forward_splits,base_features,7,"(20000, 5000)","[10, 10, 42]",175.088428,30655.957631,105.529485,0.003770,0.999901,0.999901,1.104608
7,RandomForestRegressor,cross_val,train,walk_forward_splits,base_features,8,"(20000, 5000)","[10, 10, 42]",171.150362,29292.446274,111.487959,0.003133,0.999854,0.999854,1.114029
8,RandomForestRegressor,cross_val,train,walk_forward_splits,base_features,9,"(20000, 5000)","[10, 10, 42]",123.333297,15211.102061,86.282186,0.001952,0.999848,0.999848,1.241316
9,RandomForestRegressor,cross_val,train,walk_forward_splits,base_features,10,"(20000, 5000)","[10, 10, 42]",124.283867,15446.479693,87.685026,0.001928,0.999817,0.999817,1.154617


In [54]:
cv_valid_result

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,RandomForestRegressor,cross_val,valid,walk_forward_splits,base_features,1,"(20000, 5000)","[10, 10, 42]",333.069181,1.109351e+05,140.156597,0.013068,0.716959,0.716732,1.156645
1,RandomForestRegressor,cross_val,valid,walk_forward_splits,base_features,2,"(20000, 5000)","[10, 10, 42]",446.991482,1.998014e+05,383.543477,0.033719,0.437993,0.437543,1.262350
2,RandomForestRegressor,cross_val,valid,walk_forward_splits,base_features,3,"(20000, 5000)","[10, 10, 42]",1727.544555,2.984410e+06,1094.237776,0.075346,0.082122,0.081387,1.141181
3,RandomForestRegressor,cross_val,valid,walk_forward_splits,base_features,4,"(20000, 5000)","[10, 10, 42]",7139.300111,5.096961e+07,5545.183098,0.230357,-1.543589,-1.545626,1.125212
4,RandomForestRegressor,cross_val,valid,walk_forward_splits,base_features,5,"(20000, 5000)","[10, 10, 42]",12705.285413,1.614243e+08,10298.659686,0.229923,-1.953870,-1.956236,1.040986
5,RandomForestRegressor,cross_val,valid,walk_forward_splits,base_features,6,"(20000, 5000)","[10, 10, 42]",2632.163736,6.928286e+06,2018.818808,0.034966,0.633469,0.633175,1.084377
6,RandomForestRegressor,cross_val,valid,walk_forward_splits,base_features,7,"(20000, 5000)","[10, 10, 42]",1372.244856,1.883056e+06,1077.118627,0.026814,0.975834,0.975815,1.104608
7,RandomForestRegressor,cross_val,valid,walk_forward_splits,base_features,8,"(20000, 5000)","[10, 10, 42]",486.717468,2.368939e+05,362.779097,0.010312,0.972235,0.972212,1.114029
8,RandomForestRegressor,cross_val,valid,walk_forward_splits,base_features,9,"(20000, 5000)","[10, 10, 42]",628.248750,3.946965e+05,475.903588,0.010424,0.953683,0.953646,1.241316
9,RandomForestRegressor,cross_val,valid,walk_forward_splits,base_features,10,"(20000, 5000)","[10, 10, 42]",1699.787041,2.889276e+06,1001.000410,0.016379,0.951148,0.951109,1.154617


In [55]:
# Define the results to show
tuned_comparison_lst = [cv_valid_result]

# Show the comparison table
tuned_comparison_lst_df = pd.concat([train_validation_utilities.model_comparison(results, model_info, evaluator_lst) for results in tuned_comparison_lst])
tuned_comparison_lst_df

,Model,Type,Dataset,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,RandomForestRegressor,cross_val,valid,walk_forward_splits,base_features,"[10, 10, 42]",2317.720799,1.506747e+07,1860.292781,0.064125,-0.248253,-0.249252,1.159037


# Comparison table
Visualization of model performance at various stages of train / validation

In [56]:
# Concatenate final results into Pandas dataset
final_comparison_lst_df = pd.DataFrame(pd.concat([default_comparison_lst_df, tuned_comparison_lst_df], ignore_index=True))
final_comparison_lst_df

,Model,Type,Dataset,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,RandomForestRegressor,default,valid,walk_forward_splits,base_features,"[20, 5, 42]",2370.519970,1.560697e+07,1896.375868,0.065433,-0.305707,-0.306753,0.835004
1,RandomForestRegressor,default,valid,walk_forward_splits,base_and_most_corr_features,"[20, 5, 42]",2478.345441,1.576778e+07,1990.447497,0.068993,-0.408750,-0.409878,0.770615
2,RandomForestRegressor,default,valid,walk_forward_splits,base_and_least_corr_features,"[20, 5, 42]",2648.102198,1.744722e+07,2142.194716,0.073721,-0.655070,-0.656395,0.888322
3,RandomForestRegressor,default_norm,valid,walk_forward_splits,base_features_norm,"[20, 5, 42]",4311.414051,3.407697e+07,3717.619259,0.121903,-5.011666,-5.016480,0.750290
4,RandomForestRegressor,default_norm,valid,walk_forward_splits,base_and_most_corr_features_norm,"[20, 5, 42]",5185.701605,5.522636e+07,4625.144485,0.138864,-5.399166,-5.404291,0.825756
5,RandomForestRegressor,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,"[20, 5, 42]",4707.588012,3.835252e+07,4032.876095,0.124155,-2.599675,-2.602557,0.906167
6,RandomForestRegressor,cross_val,valid,walk_forward_splits,base_features,"[10, 10, 42]",2317.720799,1.506747e+07,1860.292781,0.064125,-0.248253,-0.249252,1.159037


# Model accuracy

Since predicting the price accurately is very difficult I also saw how good the models are at predicting whether the price will go up or down in this way:

For each prediction let's consider the actual market-price, next-market-price and our predicted next-market-price (prediction).
I compute whether the current prediction is correct (1) or not (0):

$$ 
prediction\_is\_correct
= 
\begin{cases}
0 \text{ if [(market-price > next-market-price) and (market-price < prediction)] or [(market-price < next-market-price) and (market-price > prediction)]} \\
1 \text{ if [(market-price > next-market-price) and (market-price > prediction)] or [(market-price < next-market-price) and (market-price < prediction)]}
\end{cases}
$$

After that I count the number of correct prediction:
$$ 
correct\_predictions
= 
\sum_{i=0}^{total\_rows} prediction\_is\_correct
$$

Finally I compute the percentage of accuracy of the model:
$$
\\ 
accuracy 
= 
(correct\_predictions / total\_rows) 
* 100
$$

In [57]:
# Convert the pandas dataset to a PySpark dataset
best_default_pred_spark = spark.createDataFrame(best_default_predictions)
validated_pred_spark = spark.createDataFrame(cv_valid_pred)

# Compute model accuracy
default_accuracy = train_validation_utilities.model_accuracy(best_default_pred_spark)
validated_accuracy = train_validation_utilities.model_accuracy(validated_pred_spark)

# Shows whether features are normalised or not
if FEATURES_NORMALIZATION:
    NEW_CHOSEN_FEATURES_LABEL = CHOSEN_FEATURES_LABEL + "_norm"
    CHOSEN_FEATURES_LABEL = NEW_CHOSEN_FEATURES_LABEL
    
# Saving accuracy data into dataframe
accuracy_data = {
    'Model': MODEL_NAME,
    'Features': CHOSEN_FEATURES_LABEL,
    'Splitting': SPLITTING_METHOD,
    'Accuracy (default)': default_accuracy,
    'Accuracy (tuned)': validated_accuracy
}
accuracy_data_df = pd.DataFrame(accuracy_data, index=['Model'])

print(f"Percentage of correct predictions for {MODEL_NAME} with {CHOSEN_FEATURES_LABEL} and {SPLITTING_METHOD} (default): {default_accuracy:.2f}%")
print(f"Percentage of correct predictions for {MODEL_NAME} with {CHOSEN_FEATURES_LABEL} and {SPLITTING_METHOD} (tuned): {validated_accuracy:.2f}%")

Percentage of correct predictions for RandomForestRegressor with base_features and walk_forward_splits (default): 50.99%
Percentage of correct predictions for RandomForestRegressor with base_features and walk_forward_splits (tuned): 51.54%


In [58]:
# Concatenate default and tuned results
default_tuned_results = [best_default_results, cv_valid_result]
default_tuned_results_df = pd.concat([train_validation_utilities.model_comparison(results, model_info, evaluator_lst) for results in default_tuned_results])
default_tuned_results_df

,Model,Type,Dataset,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,RandomForestRegressor,default,valid,walk_forward_splits,base_features,"[20, 5, 42]",2370.519970,1.560697e+07,1896.375868,0.065433,-0.305707,-0.306753,0.835004
0,RandomForestRegressor,cross_val,valid,walk_forward_splits,base_features,"[10, 10, 42]",2317.720799,1.506747e+07,1860.292781,0.064125,-0.248253,-0.249252,1.159037


# Saving final results

In [59]:
# Save all final comparison results
final_comparison_lst_df.to_csv(ALL_MODEL_RESULTS, index=False)

In [60]:
# Save relevant results (default and tuned results)
default_tuned_results_df.to_csv(REL_MODEL_RESULTS, index=False)

In [61]:
# Saving accuracy results
accuracy_data_df.to_csv(MODEL_ACCURACY_RESULTS, index=False)

In [62]:
# Export notebook in html format (remember to save the notebook and change the model name)
if LOCAL_RUNNING:
    !jupyter nbconvert --to html 4-walk-forward-split_{MODEL_NAME}.ipynb --output 4-walk-forward-split_{MODEL_NAME} --output-dir='./exports'

d:\Documents\Repository\BDC\project\.bdc\Lib\site-packages\traitlets\traitlets.py:2548: FutureWarning: Supporting extra quotes around strings is deprecated in traitlets 5.0. You can use './exports' instead of "'./exports'" if you require traitlets >=5.
  warn(
[NbConvertApp] Converting notebook 4-walk-forward-split_RandomForestRegressor.ipynb to html
[NbConvertApp] WARNING | Alternative text is missing on 1 image(s).
[NbConvertApp] Writing 541210 bytes to exports\4-walk-forward-split_RandomForestRegressor.html
